In [ ]:
import torch
import torchphysics as tp
import math
import numpy as np
import pytorch_lightning as pl

In [ ]:
print('Tutorial zu TorchPhysics:')
print('https://torchphysics.readthedocs.io/en/latest/tutorial/tutorial_start.html')

In [ ]:
from IPython.display import Image, Math, Latex
from IPython.core.display import HTML 
Image(filename='bearing.png',width = 500, height = 250)

In [ ]:
# First define all parameters:
h_0 = 16.e-06 #m = 16 um
dh = 14e-06 #m = 14 um 
D = 0.01 #m = 10 mm 
L = np.pi*D # Länge von Gebiet
u_m = 0.26 #m/s 0.26
beta = 2.2*1e-08 # 2.2e-08 m^2/N
nu_0 = 1.5e-03 # Pa·s = 1.5 mPa·s
# lower and upper bounds of parameters
nu0 = 1.0e-03 # Viskosität
nu1 = 2.5e-03
um0 = 0.2     # Geschwindigkeit
um1 = 0.4
dh0 = 10e-6   # Spaltvariaton
dh1 = 15e-6
p_0 = 1e+5 # 1e+5 N/m^2 = 1 bar
p_rel = 0 # Relativdruck 
p_skal = 100000 #Skalierungsdruck für (-1,1) Bereich

In [ ]:
# define h:
def h(x, dh): # <- hier jetzt auch dh als input
    return h_0 + dh * torch.cos(2*x/D) # x in [0,pi*D]                               

# and compute h':
def h_x(x, dh): # <- hier jetzt auch dh als input
    return -2.0*dh/D * torch.sin(2*x/D) # x in [0,pi*D]

In [ ]:
# define the function of the viscosity.
# Here we need torch.tensors, since the function will be evaluated in the pde.
# At the beginng the model will have values close to 0, 
# therefore the viscosity will also be close to zero. 
# This will make the pde condition unstable, because we divide by nu.
# For now set values smaller then 1e-06 to 1e-06 
def nu_func(nu, p):
    out = nu * torch.exp(beta * p*p_skal)
    return torch.clamp(out, min=1e-06)

def nu_x_func(nu,p):
    out = nu* beta*p_skal*torch.exp(beta*p*p_skal)
    return out 

In [ ]:
# Variables:
x  = tp.spaces.R1('x')
nu = tp.spaces.R1('nu')
um = tp.spaces.R1('um')
dh = tp.spaces.R1('dh')
# output
p = tp.spaces.R1('p')

In [ ]:
A_x  = tp.domains.Interval(x, 0, L)
A_nu = tp.domains.Interval(nu, nu0, nu1)
A_um = tp.domains.Interval(um, um0, um1)
A_dh = tp.domains.Interval(dh, dh0, dh1)

In [ ]:
#inner_sampler = tp.samplers.AdaptiveRejectionSampler(A_x*A_nu*A_um*A_dh, n_points = 500000)
inner_sampler = tp.samplers.RandomUniformSampler(A_x*A_nu*A_um*A_dh, n_points = 2000000)
# density: 4 Punkte pro Einheitsvolumen
# Boundaries
boundary_v_sampler = tp.samplers.RandomUniformSampler(A_x.boundary*A_nu*A_um*A_dh, n_points = 100000)

In [ ]:
#tp.utils.scatter(nu*um*dh, inner_sampler, boundary_v_sampler)

In [ ]:
model = tp.models.Sequential(
    tp.models.NormalizationLayer(A_x*A_nu*A_um*A_dh),
    tp.models.FCN(input_space=x*nu*um*dh, output_space=p, hidden=(20,20,20))
)

In [ ]:
display(Math(r'h(x)\frac{d^2 \tilde{p}}{d x^2}  +\left( 3 \frac{dh}{dx} - \frac{h}{\nu} \frac{d \nu}{d x} \
    \right) \frac{d \tilde{p}}{d x} = \frac{6 u_m \nu}{p_0 h^2} \frac{d h}{d x}\quad \mbox{with} \
    \quad \tilde{p}=\frac{p}{p_{skal}} '))

In [ ]:
from torchphysics.utils import grad
# Alternativ tp.utils.grad
def pde(nu, p, x, um, dh): # <- brauchen jetzt dh und um auch als input
    # evaluate the viscosity and their first derivative
    nu = nu_func(nu,p)
    nu_x = nu_x_func(nu,p)
    # implement the PDE:
    # right hand site
    rs = 6*um*nu #<- hier jetzt um statt u_m, da deine Variable so heißt   
    #  h und h_x mit Input dh:
    h_out = h(x, dh) # nur einmal auswerten
    h_x_out = h_x(x, dh) # nur einmal auswerten
    #out = h_out * grad(grad(p,x),x)- rs*h_x_out/h_out/h_out/p_skal
    out = h_out*grad(grad(p,x),x) + (3*h_x_out -h_out/nu*nu_x)*grad(p,x) - rs*h_x_out/h_out/h_out/p_skal
    return out

In [ ]:
pde_condition = tp.conditions.PINNCondition(module=model,
                                            sampler=inner_sampler,
                                            residual_fn=pde,
                                            name='pde_condition')

In [ ]:

# Hier brauchen wir immer nur den output des modells, da die Bedingung nicht
# von nu, um oder dh abhängt.

def bc_fun(p):
    return p-p_rel 

boundary_condition = tp.conditions.PINNCondition(module = model,
                                                sampler = boundary_v_sampler,
                                                residual_fn = bc_fun,
                                                name = 'pde_bc')

In [ ]:
opt_setting = tp.solver.OptimizerSetting(torch.optim.AdamW, lr=1e-2) #SGD, LBFGS
solver = tp.solver.Solver((pde_condition, boundary_condition),optimizer_setting = opt_setting)

In [ ]:
trainer = pl.Trainer(gpus='-1' if torch.cuda.is_available() else None,
                     num_sanity_val_steps=0,
                     benchmark=True,
                     log_every_n_steps=1,
                     max_steps=1000,
                     #logger=False, zur Visualisierung im tensorboard
                     checkpoint_callback=False
                     )

trainer.fit(solver)

In [ ]:
opt_setting = tp.solver.OptimizerSetting(torch.optim.LBFGS, lr=1e-3) #SGD, LBFGS
solver = tp.solver.Solver((pde_condition, boundary_condition),optimizer_setting = opt_setting)
trainer = pl.Trainer(gpus='-1' if torch.cuda.is_available() else None,
                     num_sanity_val_steps=0,
                     benchmark=True,
                     log_every_n_steps=1,
                     max_steps=600,
                     #logger=False, zur Visualisierung im tensorboard
                     checkpoint_callback=False
                     )

trainer.fit(solver)

In [ ]:
import matplotlib.pyplot as plt
solver = solver.to('cpu')
print('nu0= ',nu0,' nu1= ',nu1)
print('dh0= ',dh0, 'dh1= ', dh1, 'm')
print('um0= ', um0, 'um1= ',um1, 'm/s')
# Parameter definieren für Plot
nu_plot = 2.0e-3 
um_plot = 0.4
dh_plot = 14.25e-06
print('Minimale Spalthöhe =',  h_0-dh_plot)
plot_sampler = tp.samplers.PlotSampler(plot_domain=A_x, n_points=600, device='cpu',
                                       data_for_other_variables={'nu':nu_plot,
                                        'um':um_plot,'dh':dh_plot})
if nu0<=nu_plot and nu_plot<=nu1 and dh0<=dh_plot and dh_plot<=dh1 and um0<=um_plot and um_plot<=um1:
    fig = tp.utils.plot(model,lambda p:p,plot_sampler)   
else:
    print('Ausserhalb des Trainingsbereiches')
print('Skalierungsfaktor = ', p_skal)
plt.savefig(f'p_{um}.png', dpi=300)

In [ ]:
import xlsxwriter
#erstellen eines Workbook Objektes mit dem Dateinamen "Gleitlager_***.xlsx"
workbook = xlsxwriter.Workbook('Gleitlager.xlsx')
worksheet = workbook.add_worksheet('Tabelle_1')

worksheet.write('Ergebnistabelle Gleitlager')
worksheet.write('nu', 'dh', 'um')

workbook.close()

In [ ]:
import winsound
frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)